# Covariance Emulator Demo notebook  

The purpose of this notebook is to give you an introduction on how to design, train, and test a neural network emulator that you want to use in a cosmological data analysis.  
One of the ingredients to modern cosmological inference is the **covariance matrix**  

This notebook is designed to be run on a typical laptop. At the end of this demo, you should be able to:
- Gain some familiarity with how neural networks are designed and trained
- TODO: fill out this list

### Step 1 - Install  

This demo uses the `CovNet` package designed by Joe Adamo to emulate covariance matrices. You'll need to download the associated github repository here: https://github.com/jadamo/CovNet. The next few cells will let you install this repo and make sure it works

In [5]:
# Change this line to wherever you downloaded the repository
%cd /home/joeadamo/Research/CovNet

# install CovNet so you can use it anywhere on your computer
# This should also install all necesary dependencies
#!python -m pip install .

/home/joeadamo/Research/CovNet


In [3]:
# run CovNet unit tests (if all of these pass, everything "should" work
!python -m unittest tests/test_enviornment.py
!python -m unittest tests/test_CovaPT.py

/home/joeadamo/Research/CovNet
/home/joeadamo/Research/CovNet/data/
.
----------------------------------------------------------------------
Ran 1 test in 0.000s

OK
...
----------------------------------------------------------------------
Ran 3 tests in 2.055s

OK


In [4]:
import CovNet